In [2]:
# here we want to extract the data from web page by using the beautifulsoup
# import the library
from bs4 import BeautifulSoup
import requests
import re
import html

In [3]:
def get_degree_courses_links():
    
    page = requests.get("https://en.didattica.unipd.it/off/2024/LM/SC")
    
    # parse the page
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # extract all the links
    links = soup.find_all('a')
    # 
    # extract the href attribute
    links = [link.get('href') for link in links]
    
    # remove all string that does not have "2024/LM" in it
    links = [link for link in links if link is not None and "2024/LM" in link]
    
    # add the prefix to the links
    prefix = 'https://en.didattica.unipd.it'
    
    links = [prefix + link for link in links]
    
    # get from index 7
    links = links[5:]
    
    return links
degree_courses_links = get_degree_courses_links()

In [4]:
def get_degree_course_units_links():
    
    # extract all link from the following page
    major_url = 'https://en.didattica.unipd.it/off/2024/LM/SC/SC2490'
    
    # request the page
    page = requests.get(major_url)
    
    # parse the page
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # extract all the links
    links = soup.find_all('a')
    
    # extract the href attribute
    links = [link.get('href') for link in links]
    
    # remove all string that does not have "/N0" in it
    links = [link for link in links if link is not None and "/N0" in link]
    
    prefix = 'https://en.didattica.unipd.it'
    
    # add the prefix to the links
    links = [prefix + link for link in links]
    
    return links

In [5]:
def get_page_body(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

## Course Degree Extraction

In [6]:
def get_degree_details(body_content):
    # find the div that has id "corso_informazioni"
    details = body_content.find_all('div', id='corso_informazioni')

    # get table 
    table = html.unescape(details[0].find_all('table'))
    trs = table[0].find_all('tr')

    trs_holder = []
    for tr in trs:
        th = tr.find_all('th')
        td = tr.find_all('td')

        if len(th) == 0 or len(td) == 0:
            continue

        th = th[0].get_text()
        td = td[0].get_text()

        # clear text from white spaces and new lines
        th = th.strip()
        td = td.strip()

        # # replace "\xa" with space
        th = th.replace("\xa0", " ")
        td = td.replace("\xa0", " ")
        # 
        # # replace "\n" with space
        th = th.replace("\n", " ")
        td = td.replace("\n", " ")

        # remove any sequence of white spaces to one white space
        th = re.sub(r'\s+', ' ', th)
        td = re.sub(r'\s+', ' ', td)

        # print((th, td))
        trs_holder.append((th, td))

    return trs_holder

In [7]:
def get_degree_description(body_content):
    # find the div that has id "corso_descrizione"
    details = body_content.find_all('div', id='corso_sua_descr')

    def remove_html_tags(text):
        clean = re.compile('<.*?>')
        text = re.sub(clean, '', text)
        return html.unescape(text)

    # get text of div that class "contenuto"
    text = details[0].find_all('div', class_='contenuto')[0].get_text()
    text = remove_html_tags(text)

    # remove any sequence of white spaces to one white space
    # text = re.sub(r'\s+', ' ', text)

    # remove starting and ending white spaces
    text = text.strip()

    return text

In [8]:
def get_course_degree_course_units(body_content):
    # find the div that has id "corso_informazioni"

    table = body_content.find_all('table', class_='elenco_af')

    trs = []

    # for each table in the page get the trs
    for t in table:
        trs.extend(t.find_all('tr'))

    # remove the first tr
    trs = trs[1:]

    data = []
    
    # iterate over the trs and get the td
    for tr in trs:
        tds = tr.find_all('td')

        if len(tds) == 0:
            continue

        # get second td as course unit code
        course_unit_code = tds[1].get_text().strip()

        # get third td as course unit name
        course_unit_name_td = tds[2]
        course_unit_name = course_unit_name_td.get_text().strip()
        course_unit_name = course_unit_name.split("\n")[0]

        # get the link of the course unit
        course_unit_link = course_unit_name_td.find_all('a')
        if len(course_unit_link) == 0:
            course_unit_link = None
        else:
            course_unit_link = 'https://en.didattica.unipd.it' + course_unit_name_td.find_all('a')[0].get('href')

        # language of the course unit (second last td)
        language = tds[-2].get_text().strip()
        language = "English" if language == "ENG" else "Italian"

        course_data = {
            "course_unit_code": course_unit_code,
            "course_unit_name": course_unit_name,
            "course_unit_link": course_unit_link,
            "language": language
        }

        data.append(course_data)

    return data

In [9]:
def get_course_degree_title_and_code(page_body):
    # get tr with class "list_cds"
    tr = page_body.find_all('tr', class_='list_cds')
    
    # get Text
    page_title = tr[0].get_text()
    
    # remove white spaces and new lines
    page_title = page_title.strip()
    
    # remove "\n" and "\xa0"
    page_title = page_title.replace("\n", " ")
    
    # just keep first letter of each word capital
    page_title = page_title.title()
    
    return page_title

In [10]:
course_degree_url = "https://en.didattica.unipd.it/off/2024/LM/SC/SC2490"

# request the page
course_degree_body = get_page_body(course_degree_url)
course_degree_title = get_course_degree_title_and_code(course_degree_body)
course_degree_details = get_degree_details(course_degree_body)
course_degree_description = get_degree_description(course_degree_body)
course_degree_courses = get_course_degree_course_units(course_degree_body)
print(course_degree_title,course_degree_details,course_degree_description,course_degree_courses)

Astrophysics And Cosmology [('Degree course type', 'Second cycle degree D.M. 270/2004'), ('Degree course code', 'SC2490'), ('Activated in', '2019/20'), ('Year of degree course structure', '2019/20'), ('Classes (Ministerial field of study codes)', 'LM-17 - Physics LM-58 - Universe sciences'), ('Degree issued', ''), ('Branch', 'Padova'), ('Language', 'English'), ('Degree course tracks', '"THEORY AND MODELLING" [001PD] "OBSERVATIONS, EXPERIMENTS AND INTERPRETATION" [002PD]'), ('Degree programme regulations', "Regulations Rector's decree and regulations, issued on 09/11/2023 with RD n. 4518/2023 Annex 1 General description of educational activities Annex 2 Study progression and proposed educational activities (course units, contact hours, credits, relevant subject areas, teaching-learning methodology, etc.)")] Characteristics and objectives
The master degree in Astrophysics and Cosmology aims at providing students with a comprehensive, up-to-date view of the main fields of modern astrophys

In [11]:
def get_degree_dataframe(url):
    key_pair = {}
    
    body = get_page_body(url)
    
    degree_title = get_course_degree_title_and_code(body)
    key_pair.update({"degree_title" : degree_title})
    
    degree_details = {key.lower().replace(" ", "_"): value for key, value in get_degree_details(body)}
    key_pair.update(degree_details)
    
    degree_description = get_degree_description(body)
    key_pair.update({"degree_description": degree_description})

    degree_courses = get_course_degree_course_units(body)
    key_pair.update({"degree_course_units":degree_courses})

    return key_pair

## Course Unit Extraction

In [12]:
def get_course_title_and_code(page_body):
    page_title = page_body.find_all('div', class_='titolopagina')
    page_title = page_title[0].get_text()
    page_title = re.findall(r'(.*)\n(.*)\n(.*)\n(.*)\n', page_title)
    page_title = page_title[0][2:4]
    return page_title

In [13]:
def get_details(body_content):
    # find the div that has class "sel_informazioni"
    details = body_content.find_all('div', class_='sel_informazioni')

    # get first table in details div
    table = html.unescape(details[0].find_all('table'))
    trs = table[0].find_all('tr')

    trs_holder = []
    for tr in trs:
        th = tr.find_all('th')[0].get_text()
        td = tr.find_all('td')[0].get_text()

        # clear text from white spaces and new lines
        th = th.strip()
        td = td.strip()

        # replace "\xa" with space
        th = th.replace("\xa0", " ")
        td = td.replace("\xa0", " ")

        # replace "\n" with space
        th = th.replace("\n", " ")
        td = td.replace("\n", " ")

        trs_holder.append((th, td))

    return trs_holder

In [14]:
def get_lecturer(body_content):
    # find the div that has class "sel_docenti"
    lecturers = body_content.find_all('div', class_='sel_docenti')

    # if is empty return "No lecturer assigned to this course unit"
    if len(lecturers) == 0:
        return None
    
    # get first table in details div
    table = html.unescape(lecturers[0].find_all('table'))

    # if is empty return "No lecturer assigned to this course unit"
    if len(table) == 0:
        return None
    
    trs = table[0].find_all('tr')

    trs_holder = []
    for tr in trs:
        tds = tr.find_all('td')
        name = tds[0].get_text().strip().replace("\xa0", " ").replace("\n", " ")
        trs_holder.append(name)
    return trs_holder


In [15]:
def get_course_organization(body_content):
    # find the div that has class "sel_docenti"
    lecturers = body_content.find_all('div', class_='sel_erogazione')

    # get first table in div
    table = lecturers[0].find_all('table')

    data = []

    trs = table[0].find_all('tr')

    # extract all th and td from trs
    for tr in trs:
        th = tr.find_all('th')[0].get_text()
        td = tr.find_all('td')[0].get_text()

        # clear text from white spaces and new lines
        th = th.strip()
        td = td.strip()

        # replace "\xa" with space
        th = th.replace("\xa0", " ")
        td = td.replace("\xa0", " ")

        # replace "\n" with space
        th = th.replace("\n", " ")
        td = td.replace("\n", " ")

        data.append((th, td))

    if len(table) == 1:
        return data

    trs = table[1].find_all('tr')

    ths = trs[0].find_all('th')
    ths_text = [th.get_text() for th in ths]

    tds = trs[1].find_all('td')
    tds_text = [td.get_text() for td in tds]

    # apply the same cleaning process to ths and tds
    ths = [th.strip().replace("\xa0", " ").replace("\n", " ") for th in ths_text]
    tds = [td.strip().replace("\xa0", " ").replace("\n", " ") for td in tds_text]

    # merge ths and tds pairs
    details = list(zip(ths, tds))

    #add all to data
    data.extend(details)

    if len(trs) > 2:
        tds = trs[2].find_all('td')
        tds_text = [td.get_text() for td in tds]

        # apply the same cleaning process to ths and tds
        ths = [th.strip().replace("\xa0", " ").replace("\n", " ") for th in ths_text]
        tds = [td.strip().replace("\xa0", " ").replace("\n", " ") for td in tds_text]

        details = list(zip(ths, tds))

        #add all to data
        data.extend(details)

    return data

In [16]:
def get_course_unit_calendar(body_content):
    # find the div that has class "sel_calendario"
    structure = body_content.find_all('div', class_='sel_calendario')
    
    if len(structure) == 0:
        return None

    # get first table in div
    table = structure[0].find_all('table')
    
    if len(table) == 0:
        return None

    data = []

    trs = table[0].find_all('tr')
    
    if len(trs) == 0:
        return None

    # extract all th and td from trs
    for tr in trs:
        th = tr.find_all('th')[0].get_text()
        td = tr.find_all('td')[0].get_text()

        # clear text from white spaces and new lines
        th = th.strip()
        td = td.strip()

        # replace "\xa" with space
        th = th.replace("\xa0", " ")
        td = td.replace("\xa0", " ")

        # replace "\n" with space
        th = th.replace("\n", " ")
        td = td.replace("\n", " ")

        data.append((th, td))

    return data

In [17]:
def get_course_syllabus(body_content):
    # find the div that has class "tabella_syllabus"
    lecturers = body_content.find_all('div', class_='tabella_syllabus')

    if len(lecturers) == 0:
        return None

    # get first table in div
    table = lecturers[0].find_all('table')
    
    if len(table) == 0:
        return None
    
    trs = table[0].find_all('tr')
    
    if len(trs) == 0:
        return None

    data = []
    # extract all th and td from trs
    for tr in trs:
        tds = tr.find_all('td')
        key = tds[0].get_text().strip().replace("\xa0", " ").replace("\n", " ")
        answer = tds[1].get_text().strip().replace("\xa0", " ")
        data.append((key, answer))

    # remove : from the keys
    data = [(key.replace(":", ""), answer) for key, answer in data]
    
    return data

In [18]:
import random

# target = random.choice(target_urls)
target = "https://en.didattica.unipd.it/off/2024/LM/SC/SC2598/000ZZ/SCP6076298/N0"
body = get_page_body(target)
title_code = get_course_title_and_code(body)
details = get_details(body)
lecturer = get_lecturer(body)
organization = get_course_organization(body)
calendar = get_course_unit_calendar(body)
syllabus = get_course_syllabus(body)
print(title_code,details,lecturer,organization,calendar,syllabus)

('WEB INFORMATION MANAGEMENT', 'SCP6076298, A.A. 2024/25') [('Degree course', 'Second cycle degree in COMPUTER SCIENCE SC2598, Degree course structure A.Y. 2021/22, A.Y. 2024/25'), ('Number of ECTS credits allocated', '6.0'), ('Type of assessment', 'Mark'), ('Course unit English denomination', 'WEB INFORMATION MANAGEMENT'), ('Department of reference', 'Department of Mathematics'), ('Mandatory attendance', 'No'), ('Language of instruction', 'English'), ('Branch', 'PADOVA'), ('Single Course unit', 'The Course unit can be attended under the option Single Course unit attendance'), ('Optional Course unit', 'The Course unit can be chosen as Optional Course unit'), ('Course unit for Erasmus students', 'The course unit can be attended by Erasmus+ and other exchange students')] ['MASSIMO MARCHIORI'] [('Period', 'First semester'), ('Year', '1st Year'), ('Teaching method', 'face to face teaching'), ('Type of hours', 'Lecture'), ('Credits', '6.0'), ('Teachinghours', '48'), ('Hours ofIndividual stu

In [19]:
def get_course_unit_dataframe(url):
    key_pair = {}
    
    body = get_page_body(url)
    
    degree_code = url.split("/")[-4]
    
    title_and_code = get_course_title_and_code(body)

    course_unit_code = title_and_code[1].split(",")[0]
    key_pair.update({"course_unit_code": course_unit_code})
    key_pair.update({"course_degree_code": degree_code})
    
    course_unit_name = title_and_code[0]
    key_pair.update({"course_unit_name": course_unit_name})
    
    course_unit_code = title_and_code[1].split(" ")[-1]
    key_pair.update({"course_unit_academic_year": course_unit_code})
   
    # course unit details contain list of tuples, get key and convert to lowercase then join with underscore
    course_unit_details = {key.lower().replace(" ", "_"): value for key, value in get_details(body)}
    key_pair.update(course_unit_details)
    
    # get course unit lecturer and if more than one concatenate with "and"
    lecturer = get_lecturer(body)
    if lecturer is not None:
        course_unit_lecturer = " and ".join(get_lecturer(body))
        key_pair.update({"course_unit_lecturer": course_unit_lecturer})
    
    # get course unit organization
    # add "course_unit_organization_" before the key
    course_unit_organization = {key.lower().replace(" ", "_"): value for key, value in get_course_organization(body)} 
    course_unit_organization = {f"course_unit_organization_{key}": value for key, value in course_unit_organization.items()}
    key_pair.update(course_unit_organization)
    
    # get calendar
    # then remove "show_course_schedule" from the dictionary

    calendar = get_course_unit_calendar(body)
    if calendar is not None:
        course_unit_organization = {key.lower().replace(" ", "_"): value for key, value in calendar} 
        course_unit_organization.pop("show_course_schedule", None)
        key_pair.update(course_unit_organization)
    
    # get syllabus
    syllabus = get_course_syllabus(body)
    if syllabus is not None:
         course_unit_syllabus = {key.lower().replace(" ", "_"): value for key, value in syllabus}  
         key_pair.update(course_unit_syllabus)

    
    # add the url of the course unit
    key_pair.update({"course_unit_url": url})
    
    return key_pair

In [21]:
import pandas as pd

degree_courses_links = get_course_unit_dataframe(target)

list_of_course_units = []
list_of_degree_courses = []

first_degree_course = degree_courses_links # [:2]
for degree_course in first_degree_course:
    degree_course_dataframe = get_degree_dataframe(degree_course)
    degree_course_code = degree_course_dataframe["degree_course_code"]
    unit_courses = degree_course_dataframe["degree_course_units"]
    degree_course_dataframe.pop("degree_course_units", None)
    list_of_course_units.append({"degree_course_code":degree_course_code,"course_units":unit_courses})
    list_of_degree_courses.append(degree_course_dataframe)

df = pd.DataFrame(list_of_degree_courses)
df.fillna("Not mentioned.")
df.to_csv("degree_courses.csv", index=False,header=True)

{'course_unit_code': 'SCP6076298',
 'course_degree_code': 'SC2598',
 'course_unit_name': 'WEB INFORMATION MANAGEMENT',
 'course_unit_academic_year': '2024/25',
 'degree_course': 'Second cycle degree in COMPUTER SCIENCE SC2598, Degree course structure A.Y. 2021/22, A.Y. 2024/25',
 'number_of_ects_credits_allocated': '6.0',
 'type_of_assessment': 'Mark',
 'course_unit_english_denomination': 'WEB INFORMATION MANAGEMENT',
 'department_of_reference': 'Department of Mathematics',
 'mandatory_attendance': 'No',
 'language_of_instruction': 'English',
 'branch': 'PADOVA',
 'single_course_unit': 'The Course unit can be attended under the option Single Course unit attendance',
 'optional_course_unit': 'The Course unit can be chosen as Optional Course unit',
 'course_unit_for_erasmus_students': 'The course unit can be attended by Erasmus+ and other exchange students',
 'course_unit_lecturer': 'MASSIMO MARCHIORI',
 'course_unit_organization_period': 'First semester',
 'course_unit_organization_year

In [62]:
#   load degree_courses.csv from ../data/degree_courses.csv
degree_courses = pd.read_csv("../data/degree_courses.csv")

degree_courses = degree_courses.fillna("Not mentioned.")

# save on the same directory
# degree_courses.to_csv("degree_courses.csv", index=False,header = True)

degree_courses

,degree_title,degree_course_type,degree_course_code,activated_in,year_of_degree_course_structure,classes_(ministerial_field_of_study_codes),degree_issued,branch,language,degree_course_tracks,degree_programme_regulations,degree_description,class_(ministerial_field_of_study_code),intra-university_with
0,Astrophysics And Cosmology,Second cycle degree D.M. 270/2004,SC2490,2019/20,2019/20,LM-17 - Physics LM-58 - Universe sciences,Not mentioned.,Padova,English,"""THEORY AND MODELLING"" [001PD] ""OBSERVATIONS, ...","Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe master deg...,Not mentioned.,Not mentioned.
1,Health Biology,Second cycle degree D.M. 270/2004,SC2751,2023/24,2023/24,Not mentioned.,Not mentioned.,Padova,Italian,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe Master's d...,LM-6 - Biology,Not mentioned.
2,Industrial Biotechnology,Second cycle degree D.M. 270/2004,SC1731,2009/10,2014/15,Not mentioned.,Not mentioned.,Padova,Italian,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe Master's D...,LM-8 - Industrial Biotechnologies,Not mentioned.
3,Chemistry,Second cycle degree D.M. 270/2004,SC1169,2008/09,2018/19,Not mentioned.,Not mentioned.,Padova,Italian,"""CHIMICA"" [001PD] ""CHEMISTRY"" [002PD] ""COMPLEX...","Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe course for...,LM-54 - Chemistry,Not mentioned.
4,Industrial Chemistry,Second cycle degree D.M. 270/2004,SC1170,2008/09,2015/16,Not mentioned.,Not mentioned.,Padova,Italian,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThis course pr...,LM-71 - Industrial chemistry,Not mentioned.
5,Computational Finance,Second cycle degree D.M. 270/2004,SC2737,2023/24,2023/24,Not mentioned.,Not mentioned.,Padova,English,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nQuantitative m...,LM-16 - Finance,Not mentioned.
6,Computer Science,Second cycle degree D.M. 270/2004,SC2598,2021/22,2021/22,Not mentioned.,Not mentioned.,Padova,English,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe master pro...,LM-18 - Computer science,Not mentioned.
7,Cybersecurity,Second cycle degree D.M. 270/2004,SC2542,2020/21,2020/21,Not mentioned.,Not mentioned.,Padova,English,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe Cybersecur...,LM-66 - Computer systems safety,Not mentioned.
8,Data Science,Second cycle degree D.M. 270/2004,SC2738,2023/24,2023/24,Not mentioned.,Not mentioned.,Padova,English,"""Data Science"" [003PD] ""BDMA"" [004PD] (The dat...","Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe Master's d...,LM Data - Data science,Not mentioned.
9,Environmental Sustainability And Education,Second cycle degree D.M. 270/2004,SC2650,2022/23,2022/23,Not mentioned.,Not mentioned.,Padova,English and Italian,Common track,"Regulations Rector's decree and regulations, i...",Characteristics and objectives\nThe course off...,LM-60 - Natural sciences,Not mentioned.


In [ ]:
df = pd.DataFrame(list_of_degree_courses)

# replace the NaN with "Not mentioned"
df = df.fillna("Not mentioned.")

df.to_csv("degree_courses.csv", index=False)
list_of_course_units

In [470]:
course_units = []
for item in list_of_course_units:
    for level in item["course_units"]:
        link = level["course_unit_link"]
        if link is not None:
            course_units.append(link)

cooorses = []
for course_unit in course_units:
    print(course_unit)
    course_unit_data = get_course_unit_dataframe(course_unit)
    cooorses.append(course_unit_data)

https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/001PD/SCP9086382/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCQ0093339/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCQ0093338/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCP9086381/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCP9086380/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCQ0093378/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCP9086342/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/002PD/SCP9086344/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCP9086348/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCN1035986/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCP7081703/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/002PD/SCN1035987/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCN1035988/N0
https://en.didattica.unipd.it/off/2024/LM/SC/SC2490/000ZZ/SCP908

In [477]:
cooorses
course_units_df = pd.DataFrame(cooorses)
course_units_df = course_units_df.fillna("Not mentioned.")

# replace "Sì" with "Yes" in mandatory_attendance
course_units_df["mandatory_attendance"] = course_units_df["mandatory_attendance"].replace("Sì", "Yes")

course_units_df.to_csv("course_units.csv", index=False)

In [475]:
# load the data
degree_courses = pd.read_csv("degree_courses.csv")
course_units = pd.read_csv("course_units.csv")
course_units

,course_unit_code,course_degree_code,course_unit_name,course_unit_academic_year,degree_course,degree_course_track,number_of_ects_credits_allocated,type_of_assessment,course_unit_english_denomination,department_of_reference,...,course_unit_url,prerequisites,target_skills_and_knowledge,examination_methods,assessment_criteria,course_unit_contents,planned_learning_activities_and_teaching_methods,additional_notes_about_suggested_reading,textbooks_(and_optional_supplementary_readings),notes
0,SCP9086382,SC2490,ADVANCED ASTROPHYSICS,2024/25,Second cycle degree in ASTROPHYSICS AND COSMOL...,THEORY AND MODELLING [001PD],6.0,Mark,ADVANCED ASTROPHYSICS,Department of Physics and Astronomy,...,https://en.didattica.unipd.it/off/2024/LM/SC/S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SCQ0093339,SC2490,ASTROPHYSICS LABORATORY 1: HIGH ENERGY INSTRUM...,2024/25,Second cycle degree in ASTROPHYSICS AND COSMOL...,Common track,6.0,Mark,ASTROPHYSICS LABORATORY 1: HIGH ENERGY INSTRUM...,Department of Physics and Astronomy,...,https://en.didattica.unipd.it/off/2024/LM/SC/S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SCQ0093338,SC2490,ASTROPHYSICS LABORATORY 1: INFRARED AND OPTICA...,2024/25,Second cycle degree in ASTROPHYSICS AND COSMOL...,Common track,6.0,Mark,ASTROPHYSICS LABORATORY 1: INFRARED AND OPTICA...,Department of Physics and Astronomy,...,https://en.didattica.unipd.it/off/2024/LM/SC/S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SCP9086381,SC2490,FUNDAMENTALS OF ASTROPHYSICS AND COSMOLOGY,2024/25,Second cycle degree in ASTROPHYSICS AND COSMOL...,Common track,6.0,Mark,FUNDAMENTALS OF ASTROPHYSICS AND COSMOLOGY,Department of Physics and Astronomy,...,https://en.didattica.unipd.it/off/2024/LM/SC/S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SCP9086380,SC2490,FUNDAMENTALS OF MODERN PHYSICS,2024/25,Second cycle degree in ASTROPHYSICS AND COSMOL...,Common track,6.0,Mark,FUNDAMENTALS OF MODERN PHYSICS,Department of Physics and Astronomy,...,https://en.didattica.unipd.it/off/2024/LM/SC/S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,SCQ1095738,SC2590,HEALTH AND ENVIRONMENT IN CIRCULAR ECONOMY,2024/25,Second cycle degree in SUSTAINABLE CHEMISTRY A...,Common track,6.0,Mark,HEALTH AND ENVIRONMENT IN CIRCULAR ECONOMY,Department of Chemical Sciences,...,https://en.didattica.unipd.it/off/2023/LM/SC/S...,"Basic knowledge of environmental science, biol...",Knowledge. WHO and EEA agree on the principle ...,Esame orale Oral exam and essay,Level of learning of all topics and their corr...,Introductory Topics.- The main environmental c...,This course includes lectures in the form of p...,All the preparatory documents for the exam wil...,NaN,NaN
748,SCQ1095721,SC2590,"PSYCHOLOGY, POLICY MAKING AND EDUCATION TO A C...",2024/25,Second cycle degree in SUSTAINABLE CHEMISTRY A...,Common track,6.0,Mark,"PSYCHOLOGY, POLICY MAKING AND EDUCATION TO A C...",Department of Chemical Sciences,...,https://en.didattica.unipd.it/off/2023/LM/SC/S...,None,The course aims at:- Providing knowledge on cu...,Written exam.,The assessment criteria will be based on:- Lev...,"- The role of heuristics, emotions, and differ...",Lectures and discussions of case studies and i...,The material for the final exam preparation wi...,"Thaler, R. H., Sunstein, C. R., Nudge: The Fin...",NaN
749,SCQ1095722,SC2590,UNDERSTANDING STATISTICS OF CIRCULAR ECONOMY,2024/25,Second cycle degree in SUSTAINABLE CHEMISTRY A...,Common track,6.0,Mark,UNDERSTANDING STATISTICS OF CIRCULAR ECONOMY,Department of Chemical Sciences,...,https://en.didattica.unipd.it/off/2023/LM/SC/S...,"Basic knowledge of math, algebra and calculus.",The aim of the course is to provide a basic un...,Written report on a specific case study workin...,Evaluation will be based on the general knowle...,• General ideas. Framing a research. Sampling....,Classroom lectures followed by computer practi...,The teacher will provide the slides and other ...,"Cicchitelli G. D’Urso